In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('max_rows', 300)
pd.set_option('max_columns', 300)
import glob
import lightgbm as lgbm
from sklearn.model_selection import KFold
import pickle

import os
import sys
sys.path.append('/home/cggyoshikawa/work/kaggle/OPVP')


In [2]:
data_dir = '../input/optiver-realized-volatility-prediction/'

In [3]:
df_test = pd.read_pickle('/home/cggyoshikawa/work/kaggle/OPVP/output/feature/nb002/test.pkl')

y_pred = df_test[['row_id']]
X_test = df_test.drop(['time_id', 'row_id'], axis=1)

In [ ]:
target = np.zeros(len(X_test))
models = []
for i in range(5):
    with open('../output/model/nb002/lgbm'+str(i)+'.pkl', 'rb') as f:
        models.append(pickle.load(f))

for model in models:
    pred = model.predict(X_test, num_iteration=model.best_iteration)
    target += pred / len(models)
    
y_pred = y_pred.assign(target=target)
y_pred

,row_id,target
0,0-4,0.001280
1,0-32,0.001399
2,0-34,0.001399


In [ ]:
# y_pred.to_csv('submission.csv',index = False)